In [1]:
from bs4 import BeautifulSoup as BS
import urllib.request
import pandas as pd
import os
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns

In [2]:
# This is a big dataset, takes a while to load

nih_full = pd.read_csv("data/nih_full.csv")

/Users/maheshrao/anaconda3/envs/capstone/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6,9,15,16,17,21,24,26,31,32,35,40,44,45,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# dont forget to specify inplace = True!

nih_full.drop(nih_full[nih_full.APPLICATION_ID.isnull()].index, inplace = True)

In [4]:
# nailed it

nih_full.APPLICATION_ID = nih_full.APPLICATION_ID.astype(int)

# now try merging

In [5]:
# NIH csvs can be downloaded from https://exporter.nih.gov/ExPORTER_Catalog.aspx
# put all csvs in folder called 'abstract_csvs' in the same directory as the notebook and run the following cells
# to import, combine files, and full data set

nih_abstracts = pd.DataFrame()
for root, dirs, files in os.walk("abstract_csvs", topdown=True):
    for file in files:
        df = pd.read_csv(os.path.join(root, file), encoding = "ISO-8859-1", low_memory = False)
        nih_abstracts = nih_abstracts.append(df)

/Users/maheshrao/anaconda3/envs/capstone/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [6]:
nih_abstracts.APPLICATION_ID = nih_abstracts.APPLICATION_ID.astype(int)

In [7]:
nih_full_abstracts = nih_full.merge(nih_abstracts, how = "left", on = "APPLICATION_ID")

In [8]:
nih_full_abstracts_proj = nih_full_abstracts.drop(nih_full_abstracts[nih_full_abstracts.FULL_PROJECT_NUM.isnull()].index)

In [9]:
full_proj_unique = nih_full_abstracts_proj.FULL_PROJECT_NUM.unique()

In [10]:
weird_grant_nums = list()
for grant in full_proj_unique:
    if len(grant) >= 12:
        if grant[12] != "-":
            weird_grant_nums.append(grant)
    else:
        weird_grant_nums.append(grant)

In [11]:
weird_activity = nih_full_abstracts_proj[nih_full_abstracts_proj.FULL_PROJECT_NUM.isin(weird_grant_nums)].ACTIVITY.unique()

In [12]:
weird_inds = nih_full_abstracts_proj[nih_full_abstracts_proj.ACTIVITY.isin(weird_activity)].index

In [13]:
nih_full_abstracts_proj_2 = nih_full_abstracts_proj.drop(weird_inds)

In [14]:
nih_full_abstracts_proj_2["grant_num"] = nih_full_abstracts_proj_2.FULL_PROJECT_NUM.str[1:12]

In [15]:
gb = nih_full_abstracts_proj_2.groupby(["grant_num", "PROJECT_END"], sort = False)

In [16]:
gb_count = gb.count()

In [22]:
gb.groups()

TypeError: 'dict' object is not callable

In [25]:
gb_filter = gb.filter(lambda x: x['ABSTRACT_TEXT'].count() == 0)

In [28]:
gb_filter.sort_values("grant_num")

,Unnamed: 0_x,ACTIVITY,ADMINISTERING_IC,APPLICATION_ID,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_END,BUDGET_START,CFDA_CODE,...,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,TOTAL_COST,TOTAL_COST_SUB_PROJECT,Unnamed: 0.1,ABSTRACT_TEXT,Unnamed: 0_y,grant_num
1540415,0,A03,AH,3000011,1.0,NaN,NaN,6/30/1986,7/1/1985,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,A03AH000859
1540416,1,A03,AH,3000012,1.0,NaN,NaN,6/30/1986,7/1/1985,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,A03AH000860
1540417,2,A03,AH,3000013,1.0,NaN,NaN,6/30/1986,7/1/1985,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,A03AH000861
1540418,3,A03,AH,3000014,1.0,NaN,NaN,6/30/1986,7/1/1985,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,A03AH000862
1540419,4,A03,AH,3000015,1.0,NaN,NaN,6/30/1986,7/1/1985,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,A03AH000863
1540420,5,A03,AH,3000016,1.0,NaN,NaN,6/30/1986,7/1/1985,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,A03AH000864
1540421,6,A03,AH,3000017,1.0,NaN,NaN,6/30/1986,7/1/1985,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,A03AH000865
1540422,7,A03,AH,3000018,1.0,NaN,NaN,6/30/1986,7/1/1985,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,A03AH000866
1540423,8,A03,AH,3000019,1.0,NaN,NaN,6/30/1986,7/1/1985,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,A03AH000867
1540424,9,A03,AH,3000020,1.0,NaN,NaN,6/30/1986,7/1/1985,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,A03AH000868


In [31]:
gb_filter.index

Int64Index([     38,     356,     364,     366,     374,     409,     418,
                421,     440,     458,
            ...
            2328196, 2328201, 2328227, 2328231, 2328270, 2328285, 2328305,
            2328306, 2328327, 2328336],
           dtype='int64', length=650875)

In [32]:
nih_full_abstracts_fill = nih_full_abstracts_proj_2.drop(gb_filter.index)

In [33]:
nih_full_abstracts_filled = nih_full_abstracts_fill.groupby(["grant_num", "PROJECT_END"])["ABSTRACT_TEXT"].transform(lambda x: x.fillna(x.mode()[0]))


ValueError: Length mismatch: Expected axis has 1255849 elements, new values have 1619080 elements

In [34]:
nih_full_abstracts_fill[nih_full_abstracts_fill["ABSTRACT_TEXT"].isnull()]

,Unnamed: 0_x,ACTIVITY,ADMINISTERING_IC,APPLICATION_ID,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_END,BUDGET_START,CFDA_CODE,...,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,TOTAL_COST,TOTAL_COST_SUB_PROJECT,Unnamed: 0.1,ABSTRACT_TEXT,Unnamed: 0_y,grant_num
385,29,U60,OH,8688351,5.0,N,6/10/2014,6/30/2015,7/1/2014,262,...,Special Emphasis Panel,NaN,NaN,9,331815.0,NaN,NaN,NaN,NaN,U60OH008460
533,177,ZIF,DC,8940192,1.0,N,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8,5348897.0,NaN,NaN,NaN,NaN,ZIFDC000001
597,241,P50,GM,8731939,5.0,N,9/3/2014,8/31/2015,9/1/2014,NaN,...,Special Emphasis Panel,5969.0,NaN,3,NaN,229105.0,NaN,NaN,NaN,P50GM103368
666,310,P30,CA,8601818,5.0,N,12/27/2013,12/31/2014,1/1/2014,NaN,...,Subcommittee I - Transistion to Independence,6409.0,NaN,40,NaN,52657.0,NaN,NaN,NaN,P30CA014236
706,350,P50,GM,8890166,5.0,N,8/29/2014,5/31/2015,6/1/2014,NaN,...,Special Emphasis Panel,8645.0,NaN,3,NaN,7764.0,NaN,NaN,NaN,P50GM107632
836,480,U60,OH,8688817,5.0,N,6/4/2014,6/30/2015,7/1/2014,262,...,Special Emphasis Panel,NaN,NaN,10,120000.0,NaN,NaN,NaN,NaN,U60OH008484
911,555,P50,GM,8737290,5.0,N,9/3/2014,8/31/2015,9/1/2014,NaN,...,Special Emphasis Panel,6813.0,NaN,3,NaN,230406.0,NaN,NaN,NaN,P50GM103297
1121,765,U54,NS,8912122,1.0,N,9/19/2014,7/31/2015,9/30/2014,NaN,...,Special Emphasis Panel,5781.0,NaN,1,NaN,398957.0,NaN,NaN,NaN,U54NS092090
1136,780,P50,HD,8596826,5.0,N,11/14/2013,11/30/2014,12/1/2013,NaN,...,Special Emphasis Panel,8888.0,NaN,8,NaN,134955.0,NaN,NaN,NaN,P50HD052117
1512,1156,UH3,MH,8910888,4.0,N,9/16/2014,7/31/2015,9/16/2014,242,...,Special Emphasis Panel,NaN,NaN,2,1612560.0,NaN,NaN,NaN,NaN,UH3MH007755


In [35]:
1255894+380037

1635931

This is what causes the mismatch:
<br>
https://stackoverflow.com/questions/31348265/group-by-in-pandas-dataframe-python

In [63]:
question_rows = nih_full_abstracts_fill[(nih_full_abstracts_fill["PROJECT_END"].isnull())]

also checked if any nan values in "grant_num" and there were not

In [64]:
question_rows[question_rows.ABSTRACT_TEXT.isnull()]

,Unnamed: 0_x,ACTIVITY,ADMINISTERING_IC,APPLICATION_ID,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_END,BUDGET_START,CFDA_CODE,...,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,TOTAL_COST,TOTAL_COST_SUB_PROJECT,Unnamed: 0.1,ABSTRACT_TEXT,Unnamed: 0_y,grant_num
533,177,ZIF,DC,8940192,1.0,N,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8,5348897.0,NaN,NaN,NaN,NaN,ZIFDC000001
597,241,P50,GM,8731939,5.0,N,9/3/2014,8/31/2015,9/1/2014,NaN,...,Special Emphasis Panel,5969.0,NaN,3,NaN,229105.0,NaN,NaN,NaN,P50GM103368
666,310,P30,CA,8601818,5.0,N,12/27/2013,12/31/2014,1/1/2014,NaN,...,Subcommittee I - Transistion to Independence,6409.0,NaN,40,NaN,52657.0,NaN,NaN,NaN,P30CA014236
706,350,P50,GM,8890166,5.0,N,8/29/2014,5/31/2015,6/1/2014,NaN,...,Special Emphasis Panel,8645.0,NaN,3,NaN,7764.0,NaN,NaN,NaN,P50GM107632
911,555,P50,GM,8737290,5.0,N,9/3/2014,8/31/2015,9/1/2014,NaN,...,Special Emphasis Panel,6813.0,NaN,3,NaN,230406.0,NaN,NaN,NaN,P50GM103297
1121,765,U54,NS,8912122,1.0,N,9/19/2014,7/31/2015,9/30/2014,NaN,...,Special Emphasis Panel,5781.0,NaN,1,NaN,398957.0,NaN,NaN,NaN,U54NS092090
1136,780,P50,HD,8596826,5.0,N,11/14/2013,11/30/2014,12/1/2013,NaN,...,Special Emphasis Panel,8888.0,NaN,8,NaN,134955.0,NaN,NaN,NaN,P50HD052117
1852,1496,P50,CA,8755685,5.0,N,9/12/2014,8/31/2015,9/1/2014,NaN,...,Special Emphasis Panel,7950.0,NaN,2,NaN,1723972.0,NaN,NaN,NaN,P50CA179546
1921,1565,U54,CA,8731833,5.0,N,8/6/2014,8/31/2015,9/1/2014,NaN,...,Special Emphasis Panel,7222.0,NaN,4,NaN,129478.0,NaN,NaN,NaN,U54CA163004
1999,1643,U01,CA,8690797,5.0,N,6/11/2014,6/30/2015,7/1/2014,NaN,...,Special Emphasis Panel,5876.0,NaN,3,NaN,401691.0,NaN,NaN,NaN,U01CA168930


# Take these out then figure out what to do with them after, they have grant numbers so probably just groupby again only by grant number and then fill with mode again

In [65]:
nih_full_abstracts_fill_2 = nih_full_abstracts_fill.drop(question_rows.index)

In [71]:
nih_full_abstracts_filled = nih_full_abstracts_fill_2.groupby(["grant_num", "PROJECT_END"])["ABSTRACT_TEXT"].transform(lambda x: x.fillna(x.mode()[0]))


KeyError: ('ABSTRACT_TEXT', 'occurred at index Unnamed: 0_x')

In [72]:
len(nih_full_abstracts_filled)

1255849

In [75]:
nih_full_abstracts_complete = nih_full_abstracts_fill_2.join(nih_full_abstracts_filled, rsuffix = "_2")

In [78]:
nih_full_abstracts_complete[(nih_full_abstracts_complete["ACTIVITY"] == "R01")&(nih_full_abstracts_complete["ABSTRACT_TEXT"].isnull())]

,Unnamed: 0_x,ACTIVITY,ADMINISTERING_IC,APPLICATION_ID,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_END,BUDGET_START,CFDA_CODE,...,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,TOTAL_COST,TOTAL_COST_SUB_PROJECT,Unnamed: 0.1,ABSTRACT_TEXT,Unnamed: 0_y,grant_num,ABSTRACT_TEXT_2
74249,73893,R01,CA,8761419,1.0,N,7/24/2014,7/31/2015,8/1/2014,396,...,NaN,A1,1,332000.0,NaN,NaN,NaN,NaN,R01CA182675,DESCRIPTION (provided by applicant): This prop...
116643,40104,R01,AA,6149811,5.0,NaN,2000-01-19T00:00:00,01/31/2001,02/01/2000,273,...,NaN,NaN,28,94296.0,NaN,NaN,NaN,NaN,R01AA000187,DESCRIPTION: This project renewal proposal re...
116654,40115,R01,AA,6168182,5.0,NaN,2000-07-27T00:00:00,07/31/2001,08/01/2000,273,...,NaN,NaN,20,235465.0,NaN,NaN,NaN,NaN,R01AA004961,Recent studies have shown that acetaldehyde an...
116659,40120,R01,AA,6168204,5.0,NaN,2000-07-20T00:00:00,07/31/2001,08/01/2000,273,...,NaN,NaN,17,240244.0,NaN,NaN,NaN,NaN,R01AA005812,DESCRIPTION:(Adapted from the Investigator's A...
116660,40121,R01,AA,6168205,5.0,NaN,2000-07-28T00:00:00,04/30/2001,05/01/2000,273,...,NaN,NaN,17,634972.0,NaN,NaN,NaN,NaN,R01AA005909,This proposal is designed around the premise t...
116689,40150,R01,AA,6168222,7.0,NaN,2000-09-19T00:00:00,08/31/2001,09/19/2000,273,...,NaN,NaN,11,226908.0,NaN,NaN,NaN,NaN,R01AA007568,DESCRIPTION: (Adapted from the Investigator's ...
116690,40151,R01,AA,6168227,5.0,NaN,2000-03-31T00:00:00,03/31/2001,04/01/2000,273,...,NaN,NaN,11,328182.0,NaN,NaN,NaN,NaN,R01AA007595,APPLICANT'S ABSTRACT: We seek to continue res...
116699,40160,R01,AA,6197292,2.0,NaN,2000-06-29T00:00:00,03/31/2001,07/01/2000,273,...,NaN,A1,13,326378.0,NaN,NaN,NaN,NaN,R01AA008003,DESCRIPTION: (Adapted from the Investigator'...
116709,40170,R01,AA,6124040,5.0,NaN,1999-11-30T00:00:00,11/30/2000,12/01/1999,273,...,NaN,NaN,11,277169.0,NaN,NaN,NaN,NaN,R01AA008169,DESCRIPTION (Adapted from the Investigator's A...
116710,40171,R01,AA,6168242,5.0,NaN,2000-08-12T00:00:00,08/31/2001,09/01/2000,273,...,NaN,NaN,10,282675.0,NaN,NaN,NaN,NaN,R01AA008275,Excessive alcohol consumption is a major threa...


In [86]:
nih_full_abstracts_complete[nih_full_abstracts_complete.grant_num == "R01CA182675"]["ABSTRACT_TEXT"][275706]

# fill looks like it worked so now replace abstract_text column with new one

'DESCRIPTION (provided by applicant): This proposal aims to define the role of the tumor suppressor p53 in the regulation of metabolism, as well as the consequences of the loss of this function in tumor pathogenesis. p53 holds the distinction of being the most frequently mutated gene in human cancers, and its inactivation is essential not only for the formation of a remarkably wide range of tumors, but also for their continued survival and proliferation. Being able to understand the mechanism by which p53 suppresses tumorigenesis has been a central objective in cancer biology, one which has important implications in the treatment of a plethora of cancer types. Among the cellular response elicited by p53, emerging evidence has indicated that metabolic modulation and senescence are crucial for tumor suppression. Tumor cells rely on re-programmed metabolism to rapidly accumulate biomass and to effectively minimize oxidative damages. This proposal focuses on malic enzyme and isocitrate deh

In [88]:
nih_full_abstracts_complete[nih_full_abstracts_complete.grant_num == "R01CA182675"]["ABSTRACT_TEXT_2"][74249]

'DESCRIPTION (provided by applicant): This proposal aims to define the role of the tumor suppressor p53 in the regulation of metabolism, as well as the consequences of the loss of this function in tumor pathogenesis. p53 holds the distinction of being the most frequently mutated gene in human cancers, and its inactivation is essential not only for the formation of a remarkably wide range of tumors, but also for their continued survival and proliferation. Being able to understand the mechanism by which p53 suppresses tumorigenesis has been a central objective in cancer biology, one which has important implications in the treatment of a plethora of cancer types. Among the cellular response elicited by p53, emerging evidence has indicated that metabolic modulation and senescence are crucial for tumor suppression. Tumor cells rely on re-programmed metabolism to rapidly accumulate biomass and to effectively minimize oxidative damages. This proposal focuses on malic enzyme and isocitrate deh

In [90]:
nih_full_abstracts_complete_2 = nih_full_abstracts_complete.drop("ABSTRACT_TEXT_2", axis = 1)

In [91]:
nih_full_abstracts_complete_2.ABSTRACT_TEXT = nih_full_abstracts_filled

In [94]:
nih_full_abstracts_complete_2 = nih_full_abstracts_complete_2.append(question_rows)

In [96]:
nih_full_abstracts_complete_2[nih_full_abstracts_complete_2.ABSTRACT_TEXT.isnull()]

,Unnamed: 0_x,ACTIVITY,ADMINISTERING_IC,APPLICATION_ID,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_END,BUDGET_START,CFDA_CODE,...,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,TOTAL_COST,TOTAL_COST_SUB_PROJECT,Unnamed: 0.1,ABSTRACT_TEXT,Unnamed: 0_y,grant_num
533,177,ZIF,DC,8940192,1.0,N,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8,5348897.0,NaN,NaN,NaN,NaN,ZIFDC000001
597,241,P50,GM,8731939,5.0,N,9/3/2014,8/31/2015,9/1/2014,NaN,...,Special Emphasis Panel,5969.0,NaN,3,NaN,229105.0,NaN,NaN,NaN,P50GM103368
666,310,P30,CA,8601818,5.0,N,12/27/2013,12/31/2014,1/1/2014,NaN,...,Subcommittee I - Transistion to Independence,6409.0,NaN,40,NaN,52657.0,NaN,NaN,NaN,P30CA014236
706,350,P50,GM,8890166,5.0,N,8/29/2014,5/31/2015,6/1/2014,NaN,...,Special Emphasis Panel,8645.0,NaN,3,NaN,7764.0,NaN,NaN,NaN,P50GM107632
911,555,P50,GM,8737290,5.0,N,9/3/2014,8/31/2015,9/1/2014,NaN,...,Special Emphasis Panel,6813.0,NaN,3,NaN,230406.0,NaN,NaN,NaN,P50GM103297
1121,765,U54,NS,8912122,1.0,N,9/19/2014,7/31/2015,9/30/2014,NaN,...,Special Emphasis Panel,5781.0,NaN,1,NaN,398957.0,NaN,NaN,NaN,U54NS092090
1136,780,P50,HD,8596826,5.0,N,11/14/2013,11/30/2014,12/1/2013,NaN,...,Special Emphasis Panel,8888.0,NaN,8,NaN,134955.0,NaN,NaN,NaN,P50HD052117
1852,1496,P50,CA,8755685,5.0,N,9/12/2014,8/31/2015,9/1/2014,NaN,...,Special Emphasis Panel,7950.0,NaN,2,NaN,1723972.0,NaN,NaN,NaN,P50CA179546
1921,1565,U54,CA,8731833,5.0,N,8/6/2014,8/31/2015,9/1/2014,NaN,...,Special Emphasis Panel,7222.0,NaN,4,NaN,129478.0,NaN,NaN,NaN,U54CA163004
1999,1643,U01,CA,8690797,5.0,N,6/11/2014,6/30/2015,7/1/2014,NaN,...,Special Emphasis Panel,5876.0,NaN,3,NaN,401691.0,NaN,NaN,NaN,U01CA168930


In [97]:
nih_full_abstracts_complete_2.groupby("grant_num")["ABSTRACT_TEXT"].transform(lambda x: x.fillna(x.mode()[0]))



IndexError: index out of bounds

In [98]:
gb_2 = nih_full_abstracts_complete_2.groupby("grant_num", sort = False)

In [99]:
gb_2_filter = gb_2.filter(lambda x: x['ABSTRACT_TEXT'].count() == 0)

In [100]:
gb_2_filter

,Unnamed: 0_x,ACTIVITY,ADMINISTERING_IC,APPLICATION_ID,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_END,BUDGET_START,CFDA_CODE,...,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,TOTAL_COST,TOTAL_COST_SUB_PROJECT,Unnamed: 0.1,ABSTRACT_TEXT,Unnamed: 0_y,grant_num
533,177,ZIF,DC,8940192,1.0,N,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8,5348897.0,NaN,NaN,NaN,NaN,ZIFDC000001
6336,5980,ZIF,HL,8940194,1.0,N,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8,26065032.0,NaN,NaN,NaN,NaN,ZIFHL000001
14558,14202,ZIF,EB,8933904,1.0,N,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8,1609751.0,NaN,NaN,NaN,NaN,ZIFEB000001
15457,15101,ZIF,CA,8938549,1.0,N,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8,118085195.0,NaN,NaN,NaN,NaN,ZIFBC000001
21603,21247,ZIF,NS,8940197,1.0,N,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8,21954202.0,NaN,NaN,NaN,NaN,ZIFNS000001
24222,23866,ZIF,EY,8938550,1.0,N,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8,9902860.0,NaN,NaN,NaN,NaN,ZIFEY000001
25685,25329,ZIF,AT,8940191,1.0,N,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8,1117482.0,NaN,NaN,NaN,NaN,ZIFAT000001
30593,30237,ZIF,GM,8938551,1.0,N,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8,154233.0,NaN,NaN,NaN,NaN,ZIFGM000001
34064,33708,ZIF,CL,8952922,1.0,N,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN,ZIFCL000001
35952,35596,ZIF,NR,8940196,1.0,N,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8,1054270.0,NaN,NaN,NaN,NaN,ZIFNR000001


In [102]:
nih_full_abstracts_complete_3 = nih_full_abstracts_complete_2.drop(gb_2_filter.index)

In [104]:
nih_full_abstracts_filled_2 = nih_full_abstracts_complete_3.groupby("grant_num")["ABSTRACT_TEXT"].transform(lambda x: x.fillna(x.mode()[0]))

In [105]:
nih_full_abstracts_complete_4 = nih_full_abstracts_complete_3

In [107]:
nih_full_abstracts_complete_4["ABSTRACT_TEXT"] = nih_full_abstracts_filled_2

In [108]:
nih_full_abstracts_complete_4[nih_full_abstracts_complete_4["ABSTRACT_TEXT"].isnull()]

,Unnamed: 0_x,ACTIVITY,ADMINISTERING_IC,APPLICATION_ID,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_END,BUDGET_START,CFDA_CODE,...,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,TOTAL_COST,TOTAL_COST_SUB_PROJECT,Unnamed: 0.1,ABSTRACT_TEXT,Unnamed: 0_y,grant_num


In [110]:
nih_full_abstracts_complete_4.shape

(1559079, 52)

In [112]:
nih_full.shape

(2321267, 49)

In [113]:
# number of rows dropped after cleaning.
# Keep these and see what to do with them or if there are any trends in them for why they don't have abstract_text

2321267-1559079

762188

In [118]:
nih_full_abstracts_complete_4.columns

Index(['Unnamed: 0_x', 'ACTIVITY', 'ADMINISTERING_IC', 'APPLICATION_ID',
       'APPLICATION_TYPE', 'ARRA_FUNDED', 'AWARD_NOTICE_DATE', 'BUDGET_END',
       'BUDGET_START', 'CFDA_CODE', 'CORE_PROJECT_NUM', 'DIRECT_COST_AMT',
       'ED_INST_TYPE', 'FOA_NUMBER', 'FULL_PROJECT_NUM', 'FUNDING_ICs',
       'FUNDING_Ics', 'FUNDING_MECHANISM', 'FY', 'IC_NAME',
       'INDIRECT_COST_AMT', 'NIH_SPENDING_CATS', 'ORG_CITY', 'ORG_COUNTRY',
       'ORG_DEPT', 'ORG_DISTRICT', 'ORG_DUNS', 'ORG_FIPS', 'ORG_IPF_CODE',
       'ORG_NAME', 'ORG_STATE', 'ORG_ZIPCODE', 'PHR', 'PI_IDS', 'PI_NAMEs',
       'PROGRAM_OFFICER_NAME', 'PROJECT_END', 'PROJECT_START', 'PROJECT_TERMS',
       'PROJECT_TITLE', 'SERIAL_NUMBER', 'STUDY_SECTION', 'STUDY_SECTION_NAME',
       'SUBPROJECT_ID', 'SUFFIX', 'SUPPORT_YEAR', 'TOTAL_COST',
       'TOTAL_COST_SUB_PROJECT', 'Unnamed: 0.1', 'ABSTRACT_TEXT',
       'Unnamed: 0_y', 'grant_num'],
      dtype='object')

In [119]:
to_drop = ['Unnamed: 0_x', 'Unnamed: 0_y', 'Unnamed: 0.1']

In [122]:
also_to_drop = ['APPLICATION_TYPE', 'ACTIVITY', 'ADMINISTERING_IC', 'SERIAL_NUMBER', 'SUPPORT_YEAR']

In [136]:
drop_these_too = ["FUNDING_Ics", "IC_NAME", 'PI_NAMEs', 'ORG_COUNTRY', 'ORG_IPF_CODE']

In [139]:
last_ones = ['STUDY_SECTION_NAME']

In [140]:
#nih_full_abstracts_complete_4.drop(to_drop, axis = 1, inplace = True)
#nih_full_abstracts_complete_4.drop(also_to_drop, axis = 1, inplace = True)
#nih_full_abstracts_complete_4.drop(drop_these_too, axis = 1, inplace = True)
nih_full_abstracts_complete_4.drop(last_ones, axis = 1, inplace = True)

In [141]:
#nih_full_abstracts_complete_4[['STUDY_SECTION', 'STUDY_SECTION_NAME']]
nih_full_abstracts_complete_4.shape

(1559079, 38)

In [143]:
# save that sucker so don't have to do this noise again!

nih_full_abstracts_complete_4.to_csv('nih_full_abstracts_complete_4.csv', index = False, chunksize = 1000)

OSError: [Errno 22] Invalid argument

In [148]:
# yeah this isn't working, try dividing it into 2 and then exporting each one individually

nih_full_abstracts_top_half = nih_full_abstracts_complete_4.head(1559079)

nih_full_abstracts_bottom_half = nih_full_abstracts_complete_4.tail(779539)

In [149]:
nih_full_abstracts_top_half.to_csv('nih_full_abstracts_top_half.csv', index = False)

OSError: [Errno 22] Invalid argument

In [152]:
[nih_1, nih_2, nih_3, nih_4, nih_5, nih_6, nih_7, nih_8, nih_9, nih_10] = np.array_split(nih_full_abstracts_complete_4, 10)

In [168]:
df_list = [nih_1, nih_2, nih_3, nih_4, nih_5, nih_6, nih_7, nih_8, nih_9, nih_10]
df_names = ["nih_1", "nih_2", "nih_3", "nih_4", "nih_5", "nih_6", "nih_7", "nih_8", "nih_9", "nih_10"]

In [170]:
# loop over the dfs to save them as individual csv

for name, df in zip(df_names, df_list):
    df.to_csv('{}.csv'.format(name), index = False)

In [164]:
nih_full_abstracts_complete_4["ABSTRACT_TEXT"]

0          PROJECT SUMMARY/ABSTRACT In addition to their ...
1          DESCRIPTION (provided by applicant): This appl...
2          Project Summary: This is a predoctoral fellows...
3          DESCRIPTION (provided by applicant): AML is a ...
4          DESCRIPTION (provided by applicant): Sub Sahar...
5          ABSTRACT Heparin-induced thrombocytopenia (HIT...
6          ?    DESCRIPTION (provided by applicant): Unde...
7          PROJECT SUMMARY The mechanisms underlying the ...
8          Abstract  Defects in cardiac excitability are ...
9            Systemic sclerosis (SSc) is a complex, multi...
10         Chronic pain affects over 100 million American...
11         Abstract Atypical speech rhythms, like those o...
12         The mitochondrial dynamism/fitness/biogenesis ...
13         ?    DESCRIPTION (provided by applicant): Diab...
14         Ischemic heart disease is a leading cause of d...
15         More than 1/3 of the world?s population is ane...
16         DESCRIPTION (

In [155]:
nih_1.to_csv('nih_1.csv', index = False)

In [144]:
round(1559079/2)

779540

In [145]:
1559079-779540

779539

In [146]:
779540+779539

1559079

In [147]:
1559079==len(nih_full_abstracts_complete_4)

True